### 1) 데이터 전처리

In [32]:
##1. 데이터 준비와 전처리   
import pandas as pd
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [33]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [34]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [35]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [36]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['user_id', 'movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding = 'ISO-8859-1')
movies.head()

,user_id,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy,NaN
1,2,Jumanji (1995),Adventure|Children's|Fantasy,NaN
2,3,Grumpier Old Men (1995),Comedy|Romance,NaN
3,4,Waiting to Exhale (1995),Comedy|Drama,NaN
4,5,Father of the Bride Part II (1995),Comedy,NaN


### 2) 분석해 봅시다.   
* ratings에 있는 유니크한 영화 개수  
* ratings에 있는 유니크한 사용자 수   
* 가장 인기있는 영화 30개 (인기순)  


In [37]:
# 사용하는 컬럼만 남겨줍니다.
using_cols = ['user_id', 'movie_id', 'title', 'genre']
movies = movies[using_cols]
movies.head(10)


,user_id,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy,NaN
1,2,Jumanji (1995),Adventure|Children's|Fantasy,NaN
2,3,Grumpier Old Men (1995),Comedy|Romance,NaN
3,4,Waiting to Exhale (1995),Comedy|Drama,NaN
4,5,Father of the Bride Part II (1995),Comedy,NaN
5,6,Heat (1995),Action|Crime|Thriller,NaN
6,7,Sabrina (1995),Comedy|Romance,NaN
7,8,Tom and Huck (1995),Adventure|Children's,NaN
8,9,Sudden Death (1995),Action,NaN
9,10,GoldenEye (1995),Action|Adventure|Thriller,NaN


In [38]:
# 유니크한 영화 수 
movies['movie_id'].nunique()

3883

In [39]:
# 유니크한 사용자 수 
movies['user_id'].nunique()

3883

In [40]:
# 가장 인기 있는 영화 30개 (인기순) 
movies_count = movies.groupby('movie_id')['user_id'].count()
movies_count.sort_values(ascending=False).head(30)

movie_id
$1,000,000 Duck (1971)                               1
Outlaw, The (1943)                                   1
Only You (1994)                                      1
Open Season (1996)                                   1
Open Your Eyes (Abre los ojos) (1997)                1
Operation Condor (Feiying gaiwak) (1990)             1
Operation Condor 2 (Longxiong hudi) (1990)           1
Operation Dumbo Drop (1995)                          1
Opportunists, The (1999)                             1
Opposite of Sex, The (1998)                          1
Ordinary People (1980)                               1
Orgazmo (1997)                                       1
Original Gangstas (1996)                             1
Original Kings of Comedy, The (2000)                 1
Orlando (1993)                                       1
Oscar and Lucinda (a.k.a. Oscar & Lucinda) (1997)    1
Otello (1986)                                        1
Othello (1952)                                       1
O

### 3) 내가 선호나는 영화를 5가지 골라서 rating에 추가해 줍시다.

In [41]:
movies['movie_id'] = movies['movie_id'].str.lower() # 검색을 쉽게하기 위해 아티스트 문자열을 소문자로 바꿔줍시다.
movies.head(10)

,user_id,movie_id,title,genre
0,1,toy story (1995),Animation|Children's|Comedy,NaN
1,2,jumanji (1995),Adventure|Children's|Fantasy,NaN
2,3,grumpier old men (1995),Comedy|Romance,NaN
3,4,waiting to exhale (1995),Comedy|Drama,NaN
4,5,father of the bride part ii (1995),Comedy,NaN
5,6,heat (1995),Action|Crime|Thriller,NaN
6,7,sabrina (1995),Comedy|Romance,NaN
7,8,tom and huck (1995),Adventure|Children's,NaN
8,9,sudden death (1995),Action,NaN
9,10,goldeneye (1995),Action|Adventure|Thriller,NaN


In [42]:
# 본인이 좋아하시는 영화 데이터로 바꿔서 추가하셔도 됩니다! 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
my_favorite = ['toy story (1995)' , 'the matrix (1999)' ,'american beauty (1999)' ,'the two popes (2019)' ,'perfume: the story of a murderer (2006)']

# 'kayoung'이라는 user_id가 위 아티스트의 영화를 30회씩 보았다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ['kayoung']*5, 'movie_id': my_favorite, 'play':[30]*5})

if not movies.isin({'user_id':['kayoung']})['user_id'].any():  # user_id에 'kayoung'이라는 데이터가 없다면
    movies = movies.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

movies.tail(10)       # 잘 추가되었는지 확인해 봅시다.


,user_id,movie_id,title,genre,play
3878,3948,meet the parents (2000),Comedy,NaN,NaN
3879,3949,requiem for a dream (2000),Drama,NaN,NaN
3880,3950,tigerland (2000),Drama,NaN,NaN
3881,3951,two family house (2000),Drama,NaN,NaN
3882,3952,"contender, the (2000)",Drama|Thriller,NaN,NaN
0,kayoung,toy story (1995),NaN,NaN,30.0
1,kayoung,the matrix (1999),NaN,NaN,30.0
2,kayoung,american beauty (1999),NaN,NaN,30.0
3,kayoung,the two popes (2019),NaN,NaN,30.0
4,kayoung,perfume: the story of a murderer (2006),NaN,NaN,30.0


### 모델에 활용하기 위한 전처리 (실습)

In [43]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = movies['user_id'].unique()
movie_id_unique = movies['movie_id'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_id_to_idx = {v:k for k,v in enumerate(movie_id_unique)}

In [44]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['kayoung'])    # 358869명의 유저 중 마지막으로 추가된 유저이니 358868이 나와야 합니다. 
print(movie_id_to_idx['toy story (1995)'])

3883
0


In [45]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_movies = movies['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_movies) == len(movies):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    movies['user_id'] = temp_user_movies   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_id_movies = movies['movie_id'].map(movie_id_to_idx.get).dropna()
if len(temp_movie_id_movies) == len(movies):
    print('movie_id column indexing OK!!')
    movies['movie_id'] = temp_movie_id_movies
else:
    print('movie_id column indexing Fail!!')

movies

user_id column indexing OK!!
movie_id column indexing OK!!


,user_id,movie_id,title,genre,play
0,0,0,Animation|Children's|Comedy,NaN,NaN
1,1,1,Adventure|Children's|Fantasy,NaN,NaN
2,2,2,Comedy|Romance,NaN,NaN
3,3,3,Comedy|Drama,NaN,NaN
4,4,4,Comedy,NaN,NaN
...,...,...,...,...,...
0,3883,0,NaN,NaN,30.0
1,3883,3883,NaN,NaN,30.0
2,3883,2789,NaN,NaN,30.0
3,3883,3884,NaN,NaN,30.0


### 4) CSR matrix를 직접 만들어 봅시다. 

In [46]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = movies['user_id'].nunique()
num_movies = movies['movie_id'].nunique()

csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.movie_id)))
csr_data

<6041x3953 sparse matrix of type '<class 'numpy.int64'>'
	with 836478 stored elements in Compressed Sparse Row format>

### 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.    
Matrix Factorization에서 쪼개진 두 feature matrix를 한번에 훈련하는것이 되지 잘 되지 않기 때문에, 한쪽 고정 -> 한쪽 학습 이것을 번갈아서 수행하는 방식임.  

In [47]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [48]:
# Implicit AlternatingLeastSquares 모델의 선언    

# 1. factors: 유저와 아이템의 벡터를 몇 차원으로 할 것인지  
# 2. Regularization : Overffitting 방지, 정규화 값 얼마나 사용할 지 결정   
# 3. use_gpu: GPU 사용 여부 
# 4. iterations: epoch와 같은 의미로 반복 학습 횟수  

als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [49]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3953x6041 sparse matrix of type '<class 'numpy.int64'>'
	with 836478 stored elements in Compressed Sparse Column format>

In [50]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

모델 학습 완료!   
   
     
      
### 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나 골라 훈련된 모델이 예측한 나의 선호도 파악해보기.
아래는   
1) 나의 벡터와 내가 좋아하는 영화 중 하나에 대한 벡터를 어떻게 만들고 있는지 확인   
2) 두 벡터를 곱하면 어떤 값이 나오는지 확인   


In [53]:
#8-7번 학습 후 벡터 뽑아내는 부분 
kayoung, toy story (1995) = user_to_idx['kayoung'], movie_to_idx['toy story (1995)']
kayoung_vector, toy story (1995)_vector = als_model.user_factors[kayoung], als_model.item_factors[toy story (1995)]

print('슝=3')

SyntaxError: invalid syntax (<ipython-input-53-2aa5444a7c3b>, line 3)

#### 퍼실님 ㅠㅠ 이부분에서 계속 syntax 에러가 나는데 아무리 수정 해보아도 해결이 되지 않네요. 등록한 제가 좋아하는 영화 제목들이 문제가 있는것 같기도하고.. 혹시 이 부분에서 같은 문제 있는 분 계시면 확인 부탁 드려요 ㅠㅠ. 감사합니다! 


In [22]:
kayoung_vector

NameError: name 'kayoung_vector' is not defined

In [54]:
toy story (1995)_vector

SyntaxError: invalid syntax (<ipython-input-54-bbf640b06488>, line 1)

In [55]:
np.dot(kayoung_vector, toy story (1995)_vector)

SyntaxError: invalid syntax (<ipython-input-55-74ee18230c9a>, line 1)

In [56]:
#AlternatingLeastSquares 클래스의 similar_items 메서드 사용하여 비슷한 영화 추천 받기. 
#나오는 값은 (movie_id, 유사도) 를 tuple로 보여줌 
favorite_movie_id = 'toy story (1995)'
movie_id = movie_id_to_idx[favorite_movie_id]
similar_movie_id = als_model.similar_items(movie_id, N=15)
similar_movie_id

[(1299, 0.0),
 (1326, 0.0),
 (1314, 0.0),
 (1315, 0.0),
 (1316, 0.0),
 (1317, 0.0),
 (1318, 0.0),
 (1319, 0.0),
 (1320, 0.0),
 (1321, 0.0),
 (1322, 0.0),
 (1323, 0.0),
 (1324, 0.0),
 (1325, 0.0),
 (3952, 0.0)]

In [57]:
#artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 
idx_to_movie_id = {v:k for k,v in movie_id_to_idx.items()}
[idx_to_movie_id[i[0]] for i in similar_movie_id]

KeyError: 3952

In [59]:
def get_similar_movie_id(movie_id_name: str):
    movie_id = movie_id_to_idx[movie_id_name]
    similar_movie_id = als_model.similar_items(movie_id)
    similar_movie_id = [idx_to_movie_id[i[0]] for i in similar_movie_id]
    return similar_movie_id

print("슝=3")

슝=3


In [60]:
get_similar_movie_id('toy story (1995)')

KeyError: 3952